<a href="https://colab.research.google.com/github/vivekkishore/machinelearning/blob/main/llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LLama index

###Query GPT with LLama index

In [ ]:
# !pip install llama-index
# !pip install llama-index-llms-azure-openai
# !pip install python-dotenv

In [51]:
llm = AzureOpenAI(
    model=azure_gpt_model_name,
    deployment_name=gpt_deployment_name,
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=api_version,
)
response = llm.complete("The sky is blue")
print(response)

Yes, that is correct. The sky appears blue during the day due to the scattering of sunlight by the Earth's atmosphere.


In [54]:
from llama_index.core.llms import ChatMessage

messages=[
  ChatMessage(role='system',content='You are funny and helpful bot'
    ),
    ChatMessage(role='user',content='why is sky blue?')
]

response=llm.chat(messages)
print(response)

assistant: Well, the sky is blue because it's trying to be cool and stand out from all the other colors. It's like the sky's way of saying, "Hey, look at me, I'm the coolest color around!" Plus, blue is just a really calming and peaceful color, so it's the perfect choice for the sky.


###Using LLama index to query list of JSON

In [ ]:
# !pip install llama-index
# !pip install llama-index-llms-azure-openai
# !pip install llama-index-embeddings-azure-openai
# !pip install sqlite-utils
# !pip install python-dotenv

In [34]:
data=[
    {'name':'charls','age':10,'city':'Newyork'},
    {'name':'bob','age':15,'city':'delhi'},
    {'name':'alice','age':20,'city':'mumbai'},
     {'name':'john','age':25,'city':'chicago'},
     {'name':'maddie','age':21,'city':'pune'}
     ]

In [35]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from dotenv import dotenv_values
config = dotenv_values(".env")


azure_openai_key = config["API_KEY"]
azure_openai_endpoint = config["ENDPOINT"]
azure_gpt_model_name=config["GPT_Model_Name"]
gpt_deployment_name=config["Deployment_Name"]
api_version=config['Model_Version']
azure_embed_model_name=config["Embed_Model_Name"]
embedding_deployment_name=config["Embed_Deployment"]

llm = AzureOpenAI(
    model=azure_gpt_model_name,
    deployment_name=gpt_deployment_name,
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=api_version,
)

embed_model = AzureOpenAIEmbedding(
    model=azure_embed_model_name,
    deployment_name=embedding_deployment_name,
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=api_version,
)

In [46]:
from llama_index.core.query_engine import JSONalyzeQueryEngine
#JSON data is being converted to SQL first
engine=JSONalyzeQueryEngine(
    list_of_dict=data,
    llm=llm,
    verbose=True,
)

In [49]:
result=engine.query("Do anyone live in city bangalore?")
result

Query: Do anyone live in city bangalore?
SQL Query: SELECT COUNT(*) FROM items WHERE city = 'bangalore'
Table Schema: {'name': <class 'str'>, 'age': <class 'int'>, 'city': <class 'str'>}
SQL Response: [{'COUNT(*)': 0}]
Response: No, no one lives in the city of Bangalore.

Response(response='No, no one lives in the city of Bangalore.', source_nodes=[], metadata={'sql_query': "SELECT COUNT(*) FROM items WHERE city = 'bangalore'", 'table_schema': "{'name': <class 'str'>, 'age': <class 'int'>, 'city': <class 'str'>}"})

In [50]:
print(result)

No, no one lives in the city of Bangalore.


###Query text file with LLamma index



In [ ]:
!pip install llama-index docx2txt pypdf python-dotenv llama-index-embeddings-azure-openai llama-index-llms-azure-openai

In [2]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from dotenv import dotenv_values

config=dotenv_values(".env")

azure_openai_key = config["API_KEY"]
azure_openai_endpoint = config["ENDPOINT"]
azure_gpt_model_name=config["GPT_Model_Name"]
gpt_deployment_name=config["Deployment_Name"]
api_version=config['Model_Version']
azure_embed_model_name=config["Embed_Model_Name"]
embedding_deployment_name=config["Embed_Deployment"]

llm = AzureOpenAI(
    model=azure_gpt_model_name,
    deployment_name=gpt_deployment_name,
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=api_version,
)

embed_model = AzureOpenAIEmbedding(
    model=azure_embed_model_name,
    deployment_name=embedding_deployment_name,
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=api_version,
)

In [14]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [30]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

documents = SimpleDirectoryReader( input_files=["sample1.txt"]).load_data()
index = VectorStoreIndex.from_documents(documents)

In [33]:
query="What is SP initiated integration?"
query_engine=index.as_query_engine(response_mode=  )
answer=query_engine.query(query)

# print(answer.get_formatted_sources())
print("query was:", query)
print("answer is:", answer)

query was: What is SP initiated integration?
answer is: SP-initiated integration is a login process in which a user navigates to the vendor application, and then the application redirects the unauthenticated user to Azure AD for authentication. After the user is authenticated, they are redirected back to the application with an authorization token.


In [59]:
from llama_index.core import PromptTemplate
template=(
    "I have provided the context below \n"
    "-------------------------\n"
    "You are a funny bot who always starts the answer with one joke \n"
    "---------------------------------- \n"
    "Given this context, please answer this question: {query_str} \n"
)

query="What is SP initiated integration?"
qa_prompt=PromptTemplate(template)

prompt=qa_prompt.format(query_str=query)
# print(prompt)

answer=query_engine.query(prompt)
print(answer)


Why did the scarecrow win an award? Because he was outstanding in his field! 

SP-initiated integration is a login process in which a user navigates to the vendor application, which then redirects the user to Azure AD for authentication. After authentication, the user is redirected back to the application with an authorization token.


###Query pdf with LLama index